In [1]:
import torch
from torch import nn
from torch.nn import functional as F

### Encoder

In [2]:
class VAE_Encoder(nn.Sequential):
    def __init__(self):
        super().__init__(
            # (B, C, H, W) -> (B, 128, H, W)
            nn.Conv2d(in_channels=3, out_channels=128, kernel_size=3, padding=1),
            # (B, 128, H, W) -> (B, 128, H, W)
            VAE_ResidualBlock(in_c=128, out_c=128),
            # (B, 128, H, W) -> (B, 128, H, W)
            VAE_ResidualBlock(in_c=128, out_c=128),
            # (B, 128, H, W) -> (B, 128, H / 2, W / 2)
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=2, padding=0),
            # (B, 128, H / 2, W / 2) -> (B, 256, H / 2, W / 2)
            VAE_ResidualBlock(in_c=128, out_c=256),
            # (B, 256, H / 2, W / 2) -> (B, 256, H / 2, W / 2)
            VAE_ResidualBlock(in_c=256, out_c=256),
            # (B, 256, H / 2, W / 2) -> (B, 256, H / 4, W / 4)
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=2, padding=0),
            # (B, 256, H / 4, W / 4) -> (B, 512, H / 4, W / 4)
            VAE_ResidualBlock(in_c=256, out_c=512),
            # (B, 512, H / 4, W / 4) -> (B, 512, H / 4, W / 4)
            VAE_ResidualBlock(in_c=512, out_c=512),
            # (B, 512, H / 4, W / 4) -> (B, 512, H / 8, W / 8)
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=2, padding=0),
            # (B, 512, H / 8, W / 8) -> (B, 512, H / 8, W / 8)
            VAE_ResidualBlock(in_c=512, out_c=512),
            VAE_ResidualBlock(in_c=512, out_c=512),
            VAE_ResidualBlock(in_c=512, out_c=512),
            # (B, 512, H / 8, W / 8) -> (B, 512, H / 8, W / 8)
            VAE_AttentionBlock(512),
            # (B, 512, H / 8, W / 8) -> (B, 512, H / 8, W / 8)
            VAE_ResidualBlock(in_c=512, out_c=512),
            # (B, 512, H / 8, W / 8) -> (B, 512, H / 8, W / 8)
            nn.GroupNorm(num_groups=32, num_channels=512),
            # (B, 512, H / 8, W / 8) -> (B, 512, H / 8, W / 8)
            nn.SiLU(),
            # (B, 512, H / 4, W / 4) -> (B, 8, H / 8, W / 8)
            nn.Conv2d(in_channels=512, out_channels=8, kernel_size=3, padding=1),
            # (B, 8, H / 4, W / 4) -> (B, 8, H / 8, W / 8)
            nn.Conv2d(in_channels=8, out_channels=8, kernel_size=1, padding=0)
        )
    
    def forward(self, x: torch.Tensor, noise: torch.Tensor) -> torch.Tensor:
        # x: (B, C, H, W) | (Image)
        # noise: (B, out_channels, H / 8, W / 8)
        for module in self:
            if getattr(module, "stride", None) == (2, 2):
                # Padding: L, R, T, B
                x = F.pad(x, (0, 1, 0, 1))
            x = module(x)
        
        # (B, 8, H / 4, W / 4) -> 2 * (B, 4, H / 8, W / 8) | Breaking the batch
        mean, log_variance = torch.chunk(x, 2, dim=1)

        # (B, 4, H / 8, W / 8) -> (B, 4, H / 8, W / 8)
        variance = torch.exp(torch.clamp(log_variance, -30, 20))
        stdev = torch.sqrt(variance)

        # Z = N(0, 1) -> N(mean, variance) = X?
        # X = mean + stdev * Z
        # Scale the output by a constant
        x = (mean + (stdev * noise)) * 0.18215

        return x

### Decoder

In [3]:
class VAE_AttentionBlock(nn.Module):
    def __init__(self, channels: int):
        super().__init__()
        self.g_norm = nn.GroupNorm(num_groups=32, num_channels=channels)
        self.attention = SelfAttention(1, channels)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x: (B, Features or channels, H, W)
        residue = x
        n, c, h, w = x.shape
        # (B, Features, H, W) -> (B, Features, H * W)
        x = x.view(n, c, h*w)
        # (B, Features, H * W) -> (B, H * W, Features)
        x = x.transpose(-1, -2)
        # (B, Features, H * W) -> (B, H * W, Features)
        x = self.attention(x)
        # (B, H * W, Features) -> (B, Features, H * W)
        x = x.transpose(-1, -2)
        # (B, Features, H * W) -> (B, Features, H, W)
        x = x.view(n, c, h, w)
        
        x += residue
        return x


In [4]:
class VAE_ResidualBlock(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()
        self.g_norm_1 = nn.GroupNorm(num_groups=32, num_channels=in_c)
        self.conv_1 = nn.Conv2d(in_channels=in_c, out_channels=out_c, kernel_size=3, padding=1)
        
        self.g_norm_2 = nn.GroupNorm(num_groups=in_c, num_channels=out_c)
        self.conv_2 = nn.Conv2d(in_channels=out_c, out_channels=out_c, kernel_size=3, padding=1)

        if in_c == out_c:
            self.residual_layer = nn.Identity()
        else:
            self.residual_layer = nn.Conv2d(in_channels=out_c, out_channels=out_c, kernel_size=1, padding=0)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x: (B, in_channels, H, W)
        residue = x
        x = self.g_norm_1(x)
        x = F.silu(x)
        x = self.conv_1(x)
        x = self.g_norm_2(x)
        x = F.silu(x)
        x = self.conv_2(x)
        return x + self.residual_layer(residue)

In [5]:
class VAE_Decoder(nn.Sequential):
    def __init__(self):
        super().__init__(
            nn.Conv2d(4, 4, kernel_size=1, padding=0),
            nn.Conv2d(4, 512, kernel_size=3, padding=1),
            VAE_ResidualBlock(512, 512),
            VAE_AttentionBlock(512),
            VAE_ResidualBlock(512, 512),
            VAE_ResidualBlock(512, 512),
            VAE_ResidualBlock(512, 512),
            # (B, 512, H / 8, W / 8) -> (B, 512, H / 8, W / 8)
            VAE_ResidualBlock(512, 512),
            # (B, 512, H / 8, W / 8) -> (B, 521, H / 4, W / 4)
            nn.Upsample(scale_factor=2),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            VAE_ResidualBlock(512, 512),
            VAE_ResidualBlock(512, 512),
            VAE_ResidualBlock(512, 512),
            # (B, 512, H / 4, W / 4) -> (B, 521, H / 2, W / 2)
            nn.Upsample(scale_factor=2),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            VAE_ResidualBlock(512, 256),
            VAE_ResidualBlock(256, 256),
            VAE_ResidualBlock(256, 256),
            # (B, 512, H / 2, W / 2) -> (B, 521, H, W)
            nn.Upsample(scale_factor=2),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            VAE_ResidualBlock(256, 128),
            VAE_ResidualBlock(128, 128),
            VAE_ResidualBlock(128, 128),
            nn.GroupNorm(32, 128),
            nn.SiLU(),
            # (B, 128, H, W) -> (B, 3, H, W)
            nn.Conv2d(128, 3, kernel_size=3, padding=1),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x /= 0.18215
        for module in self:
            x = module(x)
        return x

### Attention

In [6]:
import math
class SelfAttention(nn.Module):
    def __init__(self, n_heads: int, d_embed: int, in_proj_bias=True, out_proj_bias=True):
        super().__init__()
        self.in_proj = nn.Linear(in_features=d_embed, out_features=3 * d_embed, bias=in_proj_bias)
        self.out_proj = nn.Linear(in_features=d_embed, out_features=d_embed, bias=out_proj_bias)
        self.n_heads = n_heads
        self.d_head = d_embed // n_heads

    def forward(self, x: torch.Tensor, causual_mask=False):
        # x: (B, Seq_len, Dim)
        input_shape = x.shape
        batch_size, sequence_length, d_embed = input_shape
        interim_shape = (batch_size, sequence_length, self.n_heads, self.d_head)

        # (B, Seq_len, Dim) -> (B, Seq_len, Dim * 3) -> 3 tensors of shape (B, Seq_len, Dim)
        q, k, v = self.in_proj(x).chunk(3, dim=-1)

        # (B, Seq_len, Dim) -> (B, Seq_len, H, Dim / H) -> (B, H, Seq_len, Dim / H)
        q = q.view(interim_shape).transpose(1, 2)
        k = k.view(interim_shape).transpose(1, 2)
        v = v.view(interim_shape).transpose(1, 2)

        # (B, H, Seq_len, Seq_len)
        weight = q @ k.transpose(-1, -2)

        if causual_mask:
            # Mask where the upper triangle (above the principal diagnol) is made up of 1
            mask = torch.ones_like(weight, dtype=torch.bool).triu(1)
            weight.masked_fill(mask, -torch.inf)

        weight /= math.sqrt(self.d_head)
        weight = F.softmax(weight, dim=-1)
        # (B, H, Seq_len, Seq_len) @ (B, H, Seq_len, Dim / H) -> (B, H, Seq_len, Dim / H)
        output = weight @ v
        # (B, H, Seq_len, Dim / H) @ (B, Seq_len, H, Dim / H)
        output = output.transpose(1, 2)
        output = output.reshape(input_shape)
        output = self.out_proj(output)
        return output


In [7]:
class CrossAttention(nn.Module):
    def __init__(self, n_heads, d_embed, d_cross, in_proj_bias=True, out_proj_bias=True):
        super().__init__()
        self.q_proj   = nn.Linear(d_embed, d_embed, bias=in_proj_bias)
        self.k_proj   = nn.Linear(d_cross, d_embed, bias=in_proj_bias)
        self.v_proj   = nn.Linear(d_cross, d_embed, bias=in_proj_bias)
        self.out_proj = nn.Linear(d_embed, d_embed, bias=out_proj_bias)
        self.n_heads = n_heads
        self.d_head = d_embed // n_heads
    
    def forward(self, x, y):
        # x (latent): (B, Seq_Len_Q, Dim_Q)
        # y (context): (B, Seq_Len_KV, Dim_KV) = (B, 77, 784)
        input_shape = x.shape
        batch_size, sequence_length, d_embed = input_shape
        interim_shape = (batch_size, -1, self.n_heads, self.d_head)

        q = self.q_proj(x)
        k = self.k_proj(y)
        v = self.v_proj(y)

        q = q.view(interim_shape).transpose(1, 2)
        k = k.view(interim_shape).transpose(1, 2)
        v = v.view(interim_shape).transpose(1, 2)

        weight = q @ k.transpose(-1, -2)
        weight /= math.sqrt(self.d_head)
        weight = F.softmax(weight, dim=-1)

        output = weight @ v
        output = output.transpose(1, 2).contiguous()
        output = output.view(input_shape)
        output = self.out_proj(output)
        return output

### CLIP 

In [8]:
class CLIPEmbedding(nn.Module):
    def __init__(self, n_vocab: int, n_embed: int, n_tokens: int):
        super().__init__()
        self.token_embedding = nn.Embedding(n_vocab, n_embed)
        self.position_embedding = nn.Parameter(torch.zeros(n_tokens, n_embed))

    def forward(self, tokens):
        # (B, Seq_len) -> (B, Seq_len, Dim)
        x = self.token_embedding(tokens)
        x += self.position_embedding
        return x
        

In [9]:
class CLIPLayer(nn.Module):
    def __init__(self, n_head: int, n_embed: int):
        super().__init__()
        self.layernorm_1 = nn.LayerNorm(n_embed)
        self.attention = SelfAttention(n_head, n_embed)
        self.layernorm_2 = nn.LayerNorm(n_embed)
        self.linear_1 = nn.Linear(n_embed, 4*n_embed)
        self.linear_2 = nn.Linear(4*n_embed, n_embed)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # (B, Seq_len, Dim)
        residue = x
        
        ## SELF ATTENTION
        x = self.layernorm_1(x)
        x = self.attention(x, casual_mask=True)
        x += residue

        ## FEEDFORWARD LAYER
        residue = x
        x = self.layernorm_2(x)
        x = self.linear_1(x)
        x = x * torch.sigmoid(1.702 * x) # Quick GELU activation function
        x = self.linear_2(x)
        x += residue

        return x

In [10]:
class CLIP(nn.Module):
    def __init__(self):
        self.embedding = CLIPEmbedding(49408, 768, 77)
        self.layers = nn.ModuleList([
            CLIPLayer(12, 768) for i in range(12)
        ])
        self.layernorm = nn.LayerNorm(768)
    
    def forward(self, tokens: torch.LongTensor) -> torch.FloatTensor:
        tokens = tokens.type(torch.long)
        # (B, Seq_len) -> (B, Seq_len, Dim)
        state = self.embedding(tokens)
        for layer in self.layers:
            state = layer(state)
        # (B, Seq_len, Dim)
        output = self.layernorm(state)
        return output

## UNet

In [11]:
class TimeEmbedding(nn.Module):
    def __init__(self, n_embed: int):
        super().__init__()
        self.linear_1 = nn.Lineear(n_embed, 4*n_embed)
        self.linear_2 = nn.Linear(4 * n_embed, 4 * n_embed)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x: (1, 320)
        x = self.linear_1(x)
        x = F.silu(x)
        x = self.linear_2(x)
        # (1, 1280)
        return x

In [12]:
class UNET_ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, n_time=1280):
        super().__init__()
        self.groupnorm_feature = nn.GroupNorm(32, in_channels)
        self.conv_feature = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.linear_time = nn.Linear(n_time, out_channels)

        self.groupnorm_merged = nn.GroupNorm(32, out_channels)
        self.conv_merged = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)

        if in_channels == out_channels:
            self.residual_layer = nn.Identity()
        else:
            self.residual_layer = nn.Conv2d(in_channels, out_channels, kernel_size=1, padding=0)
    
    def forward(self, feature, time):
        # feature: (B, In_Channels, H, W)
        # time: (1, 1280)
        residue = feature
        feature = self.groupnorm_feature(feature)
        feature = F.silu(feature)
        feature = self.conv_feature(feature)
        time = F.silu(time)
        time = self.linear_time(time)
        merged = feature + time.unsqueeze(-1).unsqueeze(-1)
        merged = self.groupnorm_merged(merged)
        merged = F.silu(merged)
        merged = self.conv_merged(merged)
        return merged + self.residual_layer(residue)

In [13]:
class UNET_AttentionBlock(nn.Module):
    def __init__(self, n_head: int, n_embd: int, d_context=768):
        super().__init__()
        channels = n_head * n_embd
        
        self.groupnorm = nn.GroupNorm(32, channels, eps=1e-6)
        self.conv_input = nn.Conv2d(channels, channels, kernel_size=1, padding=0)

        self.layernorm_1 = nn.LayerNorm(channels)
        self.attention_1 = SelfAttention(n_head, channels, in_proj_bias=False)
        self.layernorm_2 = nn.LayerNorm(channels)
        self.attention_2 = CrossAttention(n_head, channels, d_context, in_proj_bias=False)
        self.layernorm_3 = nn.LayerNorm(channels)
        self.linear_geglu_1  = nn.Linear(channels, 4 * channels * 2)
        self.linear_geglu_2 = nn.Linear(4 * channels, channels)

        self.conv_output = nn.Conv2d(channels, channels, kernel_size=1, padding=0)
    
    def forward(self, x, context):
        # x: (B, Features, H, W)
        # context: (B, Seq_Len, Dim)
        residue_long = x
        x = self.groupnorm(x)
        x = self.conv_input(x)
        n, c, h, w = x.shape
        # (B, Features, H, W) -> (B, Features, H * W)
        x = x.view((n, c, h * w))
        # (B, Features, H * W) -> (B, H * W, Features)
        x = x.transpose(-1, -2)
        
        # Normalization + Self-Attention with skip connection
        # (B, H * W, Features)
        residue_short = x
        x = self.layernorm_1(x)
        x = self.attention_1(x)
        x += residue_short
        residue_short = x
        
        # Normalization + Cross-Attention with skip connection
        # (B, H * W, Features) -> (B, H * W, Features)
        x = self.layernorm_2(x)
        x = self.attention_2(x, context)
        x += residue_short
        residue_short = x
        # Normalization + FFN with GeGLU and skip connection
        # (B, H * W, Features) -> (B, H * W, Features)
        x = self.layernorm_3(x)
        # GeGLU as implemented in the original code: https://github.com/CompVis/stable-diffusion/blob/21f890f9da3cfbeaba8e2ac3c425ee9e998d5229/ldm/modules/attention.py#L37C10-L37C10
        # (B, H * W, Features) -> two tensors of shape (B, H * W, Features * 4)
        x, gate = self.linear_geglu_1(x).chunk(2, dim=-1) 
        x = x * F.gelu(gate)
        x = self.linear_geglu_2(x)
        x += residue_short
        x = x.transpose(-1, -2)
        # (B, Features, H * W) -> (B, Features, H, W)
        x = x.view((n, c, h, w))
        # Final skip connection between initial input and output of the block
        return self.conv_output(x) + residue_long

In [14]:
class Upsample(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.conv = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
    
    def forward(self, x):
        # (B, F, H, W) -> (B, F, H * 2, W * 2)
        x = F.interpolate(x, scale_factor=2, mode='nearest') 
        return self.conv(x)


In [15]:
class SwitchSequential(nn.Sequential):
    def forward(self, x: torch.Tensor, context: torch.Tensor, time: torch.Tensor) -> torch.Tensor:
        for layer in self:
            if isinstance(layer, UNET_AttentionBlock):
                x = layer(x, context)
            elif isinstance(layer, UNET_ResidualBlock):
                x = layer(x, time)
            else:
                x = layer(x)
        return x
        

In [16]:
class UNET_OutputLayer(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.groupnorm = nn.GroupNorm(32, in_channels)
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
    
    def forward(self, x):
        # x: (B, 320, H / 8, W / 8)
        x = self.groupnorm(x)
        x = F.silu(x)
        x = self.conv(x)
        
        # (B, 4, H / 8, W / 8)
        return x

In [17]:
class UNET(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoders = nn.ModuleList([
            # (B, 4, H / 8, W / 8)
            SwitchSequential(nn.Conv2d(4, 320, kernel_size=3, padding=1)),
            SwitchSequential(UNET_ResidualBlock(320, 320), UNET_AttentionBlock(8, 40)),
            SwitchSequential(UNET_ResidualBlock(320, 320), UNET_AttentionBlock(8, 40)),
            # (B, 320, H / 8, W / 8) -> (B, 320, H / 16, W / 16)
            SwitchSequential(nn.Conv2d(320, 320, kernel_size=3, stride=2, padding=1)),
            SwitchSequential(UNET_ResidualBlock(320, 640), UNET_AttentionBlock(8, 80)),
            SwitchSequential(UNET_ResidualBlock(640, 640), UNET_AttentionBlock(8, 80)),
            # (B, 640, H / 16, W / 16) -> (B, 640, H / 32, W / 32)
            SwitchSequential(nn.Conv2d(640, 640, kernel_size=3, stride=2, padding=1)),
            SwitchSequential(UNET_ResidualBlock(640, 1280), UNET_AttentionBlock(8, 160)),
            SwitchSequential(UNET_ResidualBlock(1280, 1280), UNET_AttentionBlock(8, 160)),
            # (B, 1280, H / 32, W / 32) -> (B, 1280, H / 64, W / 64)
            SwitchSequential(nn.Conv2d(1280, 1280, kernel_size=3, stride=2, padding=1)),
            SwitchSequential(UNET_ResidualBlock(1280, 1280)),
            # (B, 1280, H / 64, W / 64) -> (B, 1280, H / 64, W / 64)
            SwitchSequential(UNET_ResidualBlock(1280, 1280)),
        ])
        self.bottleneck = SwitchSequential(
            UNET_ResidualBlock(1280, 1280),
            UNET_AttentionBlock(8, 160),
            UNET_ResidualBlock(1280, 1280),
        )
        self.decoders = nn.ModuleList([
            SwitchSequential(UNET_ResidualBlock(2560, 1280)),
            SwitchSequential(UNET_ResidualBlock(2560, 1280)),
            SwitchSequential(UNET_ResidualBlock(2560, 1280), Upsample(1280)),
            SwitchSequential(UNET_ResidualBlock(2560, 1280), UNET_AttentionBlock(8, 160)),
            SwitchSequential(UNET_ResidualBlock(2560, 1280), UNET_AttentionBlock(8, 160)),
            SwitchSequential(UNET_ResidualBlock(1920, 1280), UNET_AttentionBlock(8, 160), Upsample(1280)),
            SwitchSequential(UNET_ResidualBlock(1920, 640), UNET_AttentionBlock(8, 80)),
            SwitchSequential(UNET_ResidualBlock(1280, 640), UNET_AttentionBlock(8, 80)),
            SwitchSequential(UNET_ResidualBlock(960, 640), UNET_AttentionBlock(8, 80), Upsample(640)),
            SwitchSequential(UNET_ResidualBlock(960, 320), UNET_AttentionBlock(8, 40)),
            SwitchSequential(UNET_ResidualBlock(640, 320), UNET_AttentionBlock(8, 40)),
            SwitchSequential(UNET_ResidualBlock(640, 320), UNET_AttentionBlock(8, 40)),
        ])

In [18]:
class Diffusion(nn.Module):
    def __init__(self):
        self.time_embedding = TimeEmbedding(320)
        self.unet = UNET()
        self.final = UNET_OutputLayer(320, 4)
    
    def forward(self, latent: torch.Tensor, context: torch.Tensor, time: torch.Tensor):
        # latent: (B, 4, H / 8, W / 8)
        # context: (B, Seq_len, Dim)
        # time: (1, 320)

        # (1, 320) -> (1, 1280)
        time = self.time_embedding(time)

        # (B, 4, H / 8, W / 8) -> (B, 4, H / 8, W / 8)
        output = self.unet(latent, context, time)

        # (B, 4, H / 8, W / 8)
        return output
